In [19]:
import math
import numpy as np
import pandas as pd
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

Reading and splitting the data train and test set

In [20]:
op = pd.read_csv("data/positions_xyz_128px_025.csv", delimiter=",", decimal=".", header=0)
op = op.iloc[4:]
op.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2709 entries, 4 to 2712
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   x       2709 non-null   float64
 1   y       2709 non-null   float64
 2   z       2709 non-null   float64
dtypes: float64(3)
memory usage: 63.6 KB


In [21]:
ip = pd.read_csv("data/red_ball_coordinates_128px_025.csv", delimiter=",", decimal=".", header=0)
ip.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2709 entries, 0 to 2708
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   u0      2709 non-null   float64
 1   v0      2709 non-null   float64
 2   u1      2595 non-null   float64
 3   v1      2595 non-null   float64
 4   u2      781 non-null    float64
 5   v2      781 non-null    float64
 6   u3      2062 non-null   float64
 7   v3      2062 non-null   float64
 8   u4      2707 non-null   float64
 9   v4      2707 non-null   float64
 10  u5      2323 non-null   float64
 11  v5      2323 non-null   float64
 12  u6      1737 non-null   float64
 13  v6      1737 non-null   float64
dtypes: float64(14)
memory usage: 296.4 KB


In [22]:
ipTest = ip.iloc[2000:]
opTest = op.iloc[2000:]
ipTrain = ip.iloc[:2000]
opTrain = op.iloc[:2000]

In [23]:
print(max(ipTest.v6))

127.5


Parameters for the NN

In [24]:
params = {'hidden_layer_sizes': [10, 10],
          'activation': 'relu', 'solver': 'adam',
          'alpha': 0.0, 'batch_size': 10,
          'random_state': 0, 'tol': 0.0001,
          'nesterovs_momentum': False,
          'learning_rate': 'constant',
          'learning_rate_init': 0.01,
          'max_iter': 1000, 'shuffle': True,
          'n_iter_no_change': 50, 'verbose': False}

Imputing NaN values

In [25]:
impute_it = IterativeImputer()
ipTest = impute_it.fit_transform(ipTest)
ipTrain = impute_it.fit_transform(ipTrain)

C:\Users\xande\venv\lib\site-packages\sklearn\impute\_iterative.py:800: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(
C:\Users\xande\venv\lib\site-packages\sklearn\impute\_iterative.py:800: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


Creating a neural network object with said parameters

In [26]:
net = MLPRegressor(**params)

Training Neural Net with training in- and output

In [27]:
net.fit(ipTrain, opTrain)

MLPRegressor(alpha=0.0, batch_size=10, hidden_layer_sizes=[10, 10],
             learning_rate_init=0.01, max_iter=1000, n_iter_no_change=50,
             nesterovs_momentum=False, random_state=0)

Predicting with the Neural Net converting it into a Dataframe

In [28]:
predicted = net.predict(ipTest)
predicted = pd.DataFrame(predicted, columns=['x', 'y', 'z'])


In [29]:
test_y = opTest.reset_index(drop=True)
test_y

,x,y,z
0,3.455523,1.744193,24.197860
1,3.702418,1.912496,24.046280
2,3.955057,2.101659,23.891540
3,4.212767,2.309759,23.733740
4,4.474944,2.534744,23.572870
...,...,...,...
704,5.115520,4.872968,9.123255
705,4.845301,5.128964,8.955405
706,4.577969,5.373679,8.790463
707,4.314191,5.604666,8.628434


In [30]:
print(predicted)
print(test_y)

            x         y          z
0    2.812591  2.667469  20.784197
1    2.867876  2.783092  21.169954
2    3.067982  2.616292  21.104511
3    3.305672  2.704616  20.813639
4    3.437699  3.156326  22.915002
..        ...       ...        ...
704  3.876421  4.470782   9.557587
705  3.872173  4.470837   9.564176
706  3.703584  4.473023   9.825653
707  3.818126  4.471538   9.648002
708  3.848049  4.471150   9.601592

[709 rows x 3 columns]
            x         y          z
0    3.455523  1.744193  24.197860
1    3.702418  1.912496  24.046280
2    3.955057  2.101659  23.891540
3    4.212767  2.309759  23.733740
4    4.474944  2.534744  23.572870
..        ...       ...        ...
704  5.115520  4.872968   9.123255
705  4.845301  5.128964   8.955405
706  4.577969  5.373679   8.790463
707  4.314191  5.604666   8.628434
708  3.691155  6.095557   8.246799

[709 rows x 3 columns]


# Distance calculation
Calculating the distance or deviation between the predicted and actual Dataframe -> How good are the results?

In [31]:
#predicted.compare(opTest)

In [32]:
#math.dist(predicted, opTest)

In [33]:
predicted.std()

x    3.006802
y    1.274579
z    6.431272
dtype: float64

In [34]:
#output_table = pd.DataFrame()
#
#for i in range(709):
#    for j in range(3):
#        output_table.iloc[i, j] = predicted[i, j] - opTest[i, j]

#print("Outputtabel met Euclidische afstanden:")
#print(output_table)

In [35]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
mse = mean_squared_error(test_y, predicted)
mae = mean_absolute_error(test_y, predicted)
r2 = r2_score(test_y, predicted)
print(mse)
print(r2)
print(mae)

10.382852890605704
0.41681056514913245
2.421560415360567
